### business_ML1: Работа с данными оттока:

- базовый EDA
- feature engineering
- обучение и валидация модели RandomForestClassifier



Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

### EDA

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


Соотношение мужчин и женщин в данных

In [ ]:
df['gender'].value_counts() / df.shape[0]

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

Статистики по полю TotalCharges (median, mean, std).

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace(" ", "0")

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].fillna(0))

In [ ]:
round(df['TotalCharges'].describe(), 2)

count    7043.00
mean     2279.73
std      2266.79
min         0.00
25%       398.55
50%      1394.55
75%      3786.60
max      8684.80
Name: TotalCharges, dtype: float64

Замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [ ]:
df['PhoneService'] = df['PhoneService'].replace({'Yes': 1, 'No': 0})

df['PhoneService'].value_counts()

1    6361
0     682
Name: PhoneService, dtype: int64

Замена значений поля Churn на числовые (Yes -> 1, No - 0)

In [ ]:
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

Замена значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [ ]:
for feature in ['StreamingMovies', 'StreamingTV', 'TechSupport']:
    df[feature] = df[feature].map({'Yes': 1, 'No': 0, 'No internet service': 0})

Заполнение пропусков в поле PhoneService значением 0

In [ ]:
df['PhoneService'] = df['PhoneService'].fillna(0)

Оставлю только указанный ниже список полей, удалив все другие и выведу верхние 3 строки

In [ ]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']


df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.50,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


### Pipeline

Разделение датасет на тренировочную и тестовую выборку

In [ ]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'


X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.25, random_state=197)


Сбор pipeline для поля gender

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline(steps=[
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

In [ ]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
6588,1,0
304,0,1
367,1,0
2278,1,0
6402,1,0
...,...,...
2894,1,0
4132,1,0
2515,1,0
828,0,1


Пайплайн с масштабированием для поля tenure

In [ ]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

Аналогичный преобразователь поля TotalCharges

In [ ]:
# numeric type

total_charges = Pipeline(steps=[('selector', NumberSelector(key='TotalCharges')), 
                                  ('standard', StandardScaler() )
                                  ])


Объединение

In [ ]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService', 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [ ]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', total_charges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

### Добавление RandomForestClassifier к пайплайну

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 197)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

Прогноз вероятности оттока для X_test

In [ ]:
pipeline.predict_proba(X_test)

array([[0.97, 0.03],
       [1.  , 0.  ],
       [0.98, 0.02],
       ...,
       [1.  , 0.  ],
       [0.11, 0.89],
       [0.99, 0.01]])

### Метрики качества получившейся модели (roc_auc, logloss)

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss

roc_auc_score = roc_auc_score(y_true=y_test, y_score=pipeline.predict_proba(X_test)[:, 1])
logloss = log_loss(y_true=y_test, y_pred=pipeline.predict_proba(X_test)[:, 1])

print(f'ROC AUC Score: {round(roc_auc_score, 4)}')
print(f'Logloss Score: {round(logloss, 4)}')

ROC AUC Score: 0.7949
Logloss Score: 0.7665


### Сохранение пайплайна

In [ ]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)